In [19]:
import pandas as pd

In [21]:
# upload the whole dataset for 2023 predicted tide values
file = 'https://erddap.marine.ie/erddap/tabledap/IMI-TidePrediction.csvp?time%2Clongitude%2Clatitude%2CstationID%2CWater_Level%2CWater_Level_ODM&time%3E=2023-01-01&time%3C=2024-01-01'

df = pd.read_csv(file)

In [23]:
# seperate days to date time
df['time (UTC)'] = pd.to_datetime(df['time (UTC)'])
df['hour'] = df['time (UTC)'].dt.hour
df['time'] = df['time (UTC)'].dt.time
df['day'] = df['time (UTC)'].dt.day
df['month'] = df['time (UTC)'].dt.month
df['year'] = df['time (UTC)'].dt.year

In [24]:
# check the dataset
df.head()

,time (UTC),longitude (degrees_east),latitude (degrees_north),stationID,Water_Level (metres),Water_Level_ODM (metres),hour,time,day,month,year
0,2023-01-01 00:15:00+00:00,-10.1016,53.9522,Achill_Island_MODELLED,3.35,0.92,0,00:15:00,1,1,2023
1,2023-01-01 00:20:00+00:00,-10.1016,53.9522,Achill_Island_MODELLED,3.36,0.93,0,00:20:00,1,1,2023
2,2023-01-01 00:25:00+00:00,-10.1016,53.9522,Achill_Island_MODELLED,3.37,0.94,0,00:25:00,1,1,2023
3,2023-01-01 00:30:00+00:00,-10.1016,53.9522,Achill_Island_MODELLED,3.38,0.95,0,00:30:00,1,1,2023
4,2023-01-01 00:35:00+00:00,-10.1016,53.9522,Achill_Island_MODELLED,3.38,0.95,0,00:35:00,1,1,2023


In [25]:
# get index of the troughs of the dataset
index = []

for i in range(len(df["Water_Level (metres)"])-1):
    if df["Water_Level (metres)"][i] > df["Water_Level (metres)"][i+1] <= df["Water_Level (metres)"][i+2]:
        index.append(i+1)
    else:
        continue

In [26]:
# get a dataframe of the troughs
troughdf = df.take(index)


In [28]:
#for i in range(len(troughdf["Water_Level (metres)"])):
troughdf["Water_Level (metres)"]

76         1.52
78         1.51
229        1.20
310        3.45
379        1.48
           ... 
3889243    0.98
3889246    0.96
3889335    2.65
3889397    0.85
3889400    0.83
Name: Water_Level (metres), Length: 73704, dtype: float64

In [51]:
# so with troughs, as seen below, we have multiple values for some days
# redo a filter on the troughdf
troughindex = []

for i in range(len(troughdf["Water_Level (metres)"])-1):
    if troughdf["Water_Level (metres)"].iloc[i] < troughdf["Water_Level (metres)"].iloc[i+1]:#<= troughdf["Water_Level (metres)"][i+2]:
        troughindex.append(i)
        #print(i)
    else:
        continue


In [52]:
troughdf["Water_Level (metres)"].iloc[i]

0.85

In [53]:
# check the indexes again
troughcheckdf = troughdf.take(troughindex)

troughcheckdf['date'] = troughcheckdf['time (UTC)'].dt.date

In [54]:
troughcheckdf

,time (UTC),longitude (degrees_east),latitude (degrees_north),stationID,Water_Level (metres),Water_Level_ODM (metres),hour,time,day,month,year,date
310,2023-01-02 02:05:00+00:00,-10.1016,53.9522,Achill_Island_MODELLED,3.45,1.02,2,02:05:00,2,1,2023,2023-01-02
458,2023-01-02 14:25:00+00:00,-10.1016,53.9522,Achill_Island_MODELLED,3.42,0.99,14,14:25:00,2,1,2023,2023-01-02
611,2023-01-03 03:10:00+00:00,-10.1016,53.9522,Achill_Island_MODELLED,3.57,1.14,3,03:10:00,3,1,2023,2023-01-03
760,2023-01-03 15:35:00+00:00,-10.1016,53.9522,Achill_Island_MODELLED,3.45,1.02,15,15:35:00,3,1,2023,2023-01-03
1059,2023-01-04 16:30:00+00:00,-10.1016,53.9522,Achill_Island_MODELLED,3.51,1.08,16,16:30:00,4,1,2023,2023-01-04
...,...,...,...,...,...,...,...,...,...,...,...,...
3888597,2023-12-29 01:10:00+00:00,-6.0127,52.9889,Wicklow_MODELLED,2.43,0.82,1,01:10:00,29,12,2023,2023-12-29
3888894,2023-12-30 01:55:00+00:00,-6.0127,52.9889,Wicklow_MODELLED,2.35,0.74,1,01:55:00,30,12,2023,2023-12-30
3889037,2023-12-30 13:50:00+00:00,-6.0127,52.9889,Wicklow_MODELLED,2.76,1.15,13,13:50:00,30,12,2023,2023-12-30
3889192,2023-12-31 02:45:00+00:00,-6.0127,52.9889,Wicklow_MODELLED,2.25,0.64,2,02:45:00,31,12,2023,2023-12-31


In [48]:
finaldf = troughcheckdf[['stationID','date', 'time', 'Water_Level (metres)']]

In [49]:
finaldf

,stationID,date,time,Water_Level (metres)
229,Achill_Island_MODELLED,2023-01-01,19:20:00,1.20
381,Achill_Island_MODELLED,2023-01-02,08:00:00,1.47
530,Achill_Island_MODELLED,2023-01-02,20:25:00,1.26
682,Achill_Island_MODELLED,2023-01-03,09:05:00,1.37
982,Achill_Island_MODELLED,2023-01-04,10:05:00,1.25
...,...,...,...,...
3888512,Wicklow_MODELLED,2023-12-28,18:05:00,0.57
3888808,Wicklow_MODELLED,2023-12-29,18:45:00,0.65
3888951,Wicklow_MODELLED,2023-12-30,06:40:00,0.88
3889104,Wicklow_MODELLED,2023-12-30,19:25:00,0.74


In [50]:
finaldf.to_excel('2023lowtides.xlsx', sheet_name= '2023')

## Everyting above is was used to complete the task
## everything below was used as practice

In [40]:
lasttry = troughcheckdf.groupby(["stationID", "month", "day","time (UTC)"])["Water_Level (metres)"].count().reset_index()

lasttry.tail(10)

,stationID,month,day,time (UTC),Water_Level (metres)
21784,Wicklow_MODELLED,12,25,2023-12-25 15:55:00+00:00,1
21785,Wicklow_MODELLED,12,26,2023-12-26 04:10:00+00:00,1
21786,Wicklow_MODELLED,12,26,2023-12-26 16:40:00+00:00,1
21787,Wicklow_MODELLED,12,27,2023-12-27 04:45:00+00:00,1
21788,Wicklow_MODELLED,12,27,2023-12-27 17:20:00+00:00,1
21789,Wicklow_MODELLED,12,28,2023-12-28 18:05:00+00:00,1
21790,Wicklow_MODELLED,12,29,2023-12-29 18:45:00+00:00,1
21791,Wicklow_MODELLED,12,30,2023-12-30 06:40:00+00:00,1
21792,Wicklow_MODELLED,12,30,2023-12-30 19:25:00+00:00,1
21793,Wicklow_MODELLED,12,31,2023-12-31 07:15:00+00:00,1


In [41]:
lasttry.groupby(["stationID", "month", "day"])["Water_Level (metres)"].count()

stationID               month  day
Achill_Island_MODELLED  1      1      2
                               2      2
                               3      1
                               4      2
                               5      2
                                     ..
Wicklow_MODELLED        12     27     2
                               28     1
                               29     1
                               30     2
                               31     1
Name: Water_Level (metres), Length: 13076, dtype: int64

In [ ]:
lasttry['time'] = lasttry['time (UTC)'].dt.time
lasttry['day'] = lasttry['time (UTC)'].dt.day
lasttry['month'] = lasttry['time (UTC)'].dt.month
lasttry['year'] = lasttry['time (UTC)'].dt.year
lasttry['date'] = lasttry['time (UTC)'].dt.date

In [ ]:
datecolumn= lasttry.select_dtypes(include=['datetime64[ns]']).columns
for date_column in datecolumn:
    df[date_column] = df[date_column].dt.date
    


In [ ]:
finaldf = lasttry[['stationID','date', 'time']]

In [ ]:
# Extract to excel
finaldf.to_excel('lowtide2023stations.xlsx', sheet_name= '2023')

## issues with trough dataset
- more than one value per day
    - how do i check - how many are in each day
    - for each station and each month and each day count the number of water_levels

In [ ]:
checkdf = troughdf.groupby(["stationID", "month", "day","time (UTC)"])["Water_Level (metres)"].count().reset_index()

In [ ]:
checkdf.tail(50)

,stationID,month,day,time (UTC),Water_Level (metres)
73654,Wicklow_MODELLED,12,23,2023-12-23 13:45:00+00:00,1
73655,Wicklow_MODELLED,12,23,2023-12-23 13:55:00+00:00,1
73656,Wicklow_MODELLED,12,23,2023-12-23 14:05:00+00:00,1
73657,Wicklow_MODELLED,12,23,2023-12-23 20:45:00+00:00,1
73658,Wicklow_MODELLED,12,23,2023-12-23 20:55:00+00:00,1
73659,Wicklow_MODELLED,12,24,2023-12-24 02:25:00+00:00,1
73660,Wicklow_MODELLED,12,24,2023-12-24 02:35:00+00:00,1
73661,Wicklow_MODELLED,12,24,2023-12-24 09:30:00+00:00,1
73662,Wicklow_MODELLED,12,24,2023-12-24 14:45:00+00:00,1
73663,Wicklow_MODELLED,12,24,2023-12-24 14:55:00+00:00,1
